The code below was used to collect airline tweets

In [ ]:
from twython import TwythonStreamer, TwythonError, TwythonRateLimitError, TwythonAuthError
from unidecode import unidecode
import sys
import json
from time import sleep
import time


tweets = []

class MyStreamer(TwythonStreamer):
    '''our own subclass of TwythonStremer'''

    # overriding
    def on_success(self, data):
        try:
            if data['lang'] == 'en':
                test = data['text']
                if not test.startswith('RT'):
                    tweets.append(unidecode(data['text']))
                    print 'received tweet #', len(tweets), unidecode(data['text'])[:100]
                
        except:
            print 'Error encountered'
        
#==============================================================================
        if len(tweets) % 10 == 0:
            self.store_json()
            #sleep(30)
            #self.disconnect()

#==============================================================================
        
#==============================================================================
        if len(tweets) >= 10000:
              self.store_json()
              self.disconnect()
              return False
#==============================================================================
    # overriding
    def on_error(self, status_code, data):
        print status_code, data
        #self.disconnect()


#==============================================================================
    def store_json(self):
        t=time.localtime()
        timestamp = time.strftime('%b_%d_%Y',t)
        #with open('tweet_stream_{}_{}.json'.format('Airlines', timestamp), 'w') as f:
        with open('tweet_stream_airlines_{}.json'.format(timestamp), 'w') as f:    
            json.dump(tweets, f, indent=4)
  
#==============================================================================

if __name__ == '__main__':

    CONSUMER_KEY = 'Input_Consumer_Key    CONSUMER_SECRET = 'Input_Consumer_Secret    ACCESS_TOKEN = 'Input_Access_Token    ACCESS_TOKEN_SECRET = 'Input_Access_Token_Secret
    stream = MyStreamer(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

#==============================================================================
    if len(sys.argv) > 1:
        keyword = sys.argv[1]
    else:
        keyword = '@AlaskaAir, @AmericanAir, @DeltaAssist, @FlyFrontier, @Jetblue, @SouthwestAir, @United, @VirginAmerica'
        #keyword = 'Chicago'
#==============================================================================

        
#    stream.statuses.filter(track=keyword)
#import sys #Do this if you want to log error output
    timeout = time.time() + 60 * 1 #minutes from now
    test = -1
    while True:  #Endless loop: personalize to suit your own purposes
        
        if test == 0 or time.time() > timeout:
            stream.store_json()
            break
        
        test = test - 1
        
        try: 
            stream.statuses.filter(track=keyword)
            #stream.statuses.filter(locations="-87.51,41.46,-87.33,42.02")
        except:
            e = sys.exc_info()[0]  #Get exception info (optional)
            print 'ERROR:',e  #Print exception info (optional)
            continue

The code below filter the tweets
- get rid of punctuation and non essential Tweets like ones about the Cuba flight. 

In [ ]:

import time
import json
from textblob import TextBlob
import string
import re

# open a text file
json_data = open('tweet_stream_Airlines_Dec_04_20165k.json').read()
data = json.loads(json_data)

######## pull out the extra characters
###### take out characters
#####string out the cuba tweets

comment = []
for line in data:
    if "Cuba" in line:
        continue
    if "@djsnake" in line:
        continue
    if "havana" in line:
        continue
    else:
        ln=line.lstrip()
        ln=line.rstrip()
        ln=re.sub("[^_a-zA-Z0-9]", ' ', line)
        comment.append(ln)
print(comment)


#######creating new file      
keyword="airline"
t=time.localtime()
timestamp = time.strftime('%b_%d_%Y',t)
        #with open('tweet_stream_{}_{}.json'.format('Airlines', timestamp), 'w') as f:
with open('airlineTweetPull_{}_{}.json'.format(keyword, timestamp), 'w') as f:    
    json.dump(comment, f, indent=4)

Predictive model - trained to predict the sentiment of Tweets (positive, negative and neutral)

In [ ]:
import nltk
import time
import json


########create tweets to train on
pos_tweets = [('I love this car', 'positive'),
('This view is amazing', 'positive'),
('I feel great this morning', 'positive'),
('I am so excited and happy about the concert', 'positive'),
('He is my awesome best friend', 'positive'),
('just had the smoothest landing I\'ve ever felt. And I\'ve flown a lot. Super impressed with @AmericanAir ', 'positive'),
('This is a shout out to @Uber, @united &amp; @Enterprise, who have all been amazing this morning.','positive'),
('now airborne. Weather permitting','positive'),
('Shoutout','positive'),
('I m loving this  so freaking awesome    great job','positive'),
('I will say this  your military friendly kiosks made it a little easier to check in','positive')
]


neg_tweets = [('I do not like this car', 'negative'),
('This view is horrible', 'negative'),
('I feel tired this morning', 'negative'),
('I am not looking forward to the concert', 'negative'),
('He is my enemy', 'negative'),
('this is not a regular delay reason  irresponsability of the crew Im just really upset  we should be compensated','negative'),
('ticket agent refused to even check status of upgrade   gate agent said ticketing should have done it','negative'),
('united Fifth flight on your airline in the past six weeks that has been late   you d think I d learn ', 'negative'),
('or you can reinstate my ticket  the one you stole  and put me in a hotel room','negative'),
('worried about the luggage being damaged underneath the plane','negative'),
('AmericanAir terrible','negative'),
('we only missed our flight because we were denied priority check in','negative')
]


neu_tweets = [('for having @djsnake #Encore album on demand', 'neutral'),
('I\'m 2 trips short of a list....Gotta work on that', 'neutral'),
('just landed', 'neutral'),
('Happy Holidays from  AmericanAir', 'neutral')
              ('@USAirways I think it\'s ok.', 'neutral'),
('@JetBlue Has DC Thinking About Summer With New Nantucket Service','neutral'),
('@united got it, look forward to hearing from you soon','neutral'),
('@JetBlue Airways to continue various commercial relationships with #Lufthansa','neutral')
]

#########
tweets = []
for (words, sentiment) in pos_tweets + neg_tweets +neu_tweets:
        words_filtered = [e.lower() for e in words.split() if len(e) >=3] 
        tweets.append((words_filtered, sentiment))

#######this gets the frequency of words in postivie and negative
def get_words_in_tweets(tweets):
    all_words = []
    for (words, sentiment) in tweets:
        all_words.extend(words)
    return all_words
def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features
    
word_features = get_word_features(get_words_in_tweets(tweets))

#print(word_features)
#feature extractor returns a dictionary indicating what words


def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features

######training and classifier
training_set = nltk.classify.apply_features(extract_features,tweets)

classifier = nltk.NaiveBayesClassifier.train(training_set)


#tweet="Bob is awesome"


#print(classifier.classify(extract_features(tweet.split())))

json_data = open('airlinetweetpull_airline_Dec_06_2016.json').read()
data = json.loads(json_data)

sentiment=[]
for l in data:
    classificaiton = classifier.classify(extract_features(l.split()))
    sentiment.append((l, classificaiton))
print(sentiment)
    

keyword="sentiment"
t=time.localtime()
timestamp = time.strftime('%b_%d_%Y',t)
        #with open('tweet_stream_{}_{}.json'.format('Airlines', timestamp), 'w') as f:
with open('{}_{}.json'.format(keyword, timestamp), 'w') as f:    
    json.dump(sentiment, f, indent=4)


The following code created our plane wordcloud

In [ ]:
from os import path
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

d = path.dirname('/Users/shannonlamberson/Dropbox/00_public1/FinalProject/')

# Read the whole text.
text = open(path.join(d,'Plane.txt')).read()

# read the mask / color image
# taken from http://jirkavinse.deviantart.com/art/quot-Real-Life-quot-Alice-282261010
alice_coloring = np.array(Image.open(path.join(d, "airplane.png")))
stopwords = set(STOPWORDS)
stopwords.add("said")

wc = WordCloud(background_color="white", max_words=2000, mask=alice_coloring,
               stopwords=stopwords, max_font_size=60, random_state=42)
# generate word cloud
wc.generate(text)

# create coloring from image
image_colors = ImageColorGenerator(alice_coloring)

# show
plt.imshow(wc)
plt.axis("off")
plt.figure()
# recolor wordcloud and show
# we could also give color_func=image_colors directly in the constructor
plt.imshow(wc.recolor(color_func=image_colors))
plt.axis("off")
plt.figure()
plt.imshow(alice_coloring, cmap=plt.cm.gray)
plt.axis("off")
plt.show()

In [ ]:
The following code gives sentiment

In [ ]:
import collections
import string
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import operator
import string
import nltk
import collections
import string
from textblob import TextBlob
import string

# open a text file
f = open('/Users/shannonlamberson/Dropbox/00_public1/FinalProject/Plane.txt')
lines = f.read().split(',')
f.close()

p= string.punctuation
d= string.digits

table_p = string.maketrans(p, len(p) * " ")
table_d = string.maketrans(d, len(d) * " ")

wordsplit = []
for ln in lines:
    line=ln.lstrip()
    line=line.translate(table_p)
    line=line.translate(table_d)
    words = line.strip().split()
    for word in words:
         if len(word)>6:
             wordsplit.append(word)
    else:
        pass
#print(wordsplit)
tb=TextBlob(str(wordsplit))
print tb.sentiment

The following code created the word frequency chart

In [ ]:
wordcount={}
#for word in file.read().split():
for word in wordsplit:
    if len(word)>6:
        if word not in wordcount:
            wordcount[word] = 1
        else:
            wordcount[word] += 1
            
#print(wordcount)

freq = nltk.FreqDist(wordcount)
freq.plot(30)



These are commands that we used to create variables.


In [ ]:
These are commands that we used to create variables.
tabulate airline, gen(airline_dummy)  # this line generated 6 numeric dummy vaiables out of the Airline string data

rename airline_dummy1 American_dum

rename airline_dummy2 delta_dum

rename airline_dummy3 southwest_dum

rename airline_dummy4 usairways_dum

rename American_dum american_dum

rename airline_dummy5 united_dum

rename airline_dummy6 virgin_dum


tabulate airline_sentiment, gen(sentiment_dummy) # this created 3 dummy variables from the Sentiment sting data

Sentiment dummy is 3 = Positive, 2 = Neutral, 1 = Negative
i.e. sentiment_dummy3
Below, we used  an indicator variable to allow all sentiment to repesenten numerically in one column
gen sent_indicator = 0
replace sent_indicator =1 if sentiment_dummy1==1
replace sent_indicator =2 if sentiment_dummy2==1
replace sent_indicator =3 if sentiment_dummy3==1


